# Chroma demo

This notebook allows for the creation of proteins in the shape of letters. Feel free to adjust the parameters and generate a protein in the first letter of your name!

In [ ]:
#Install condacolab to battle the dependency issues
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#Optional cell to check
#When this cell fails, run it again, I have no idea why this error occurs but it does, and just by running it again it works
import condacolab
condacolab.check()

In [ ]:
#Make the environment
!conda create --name chromademo python=3.9

In [ ]:
#Shell cell to install chroma
%%shell
eval "$(conda shell.bash hook)"
conda activate chromademo
pip install git+https://github.com/generatebio/chroma.git
python --version

In [ ]:
#General imports needed for chroma
import sys
import os
import contextlib
import torch
from pathlib import Path

#Demo of chroma set preferred encoding, so for safety sake
import locale
locale.getpreferredencoding = lambda: "UTF-8"

#Google collab can't find the conda environment, so added to path
sys.path.append("//usr//local//envs//chromademo//lib//python3.9//site-packages")

#Chroma imports as described in demo
from chroma import Chroma, Protein, conditioners
from chroma.models import graph_classifier, procap
from chroma.utility.api import register_key
from chroma.utility.chroma import letter_to_point_cloud, plane_split_protein

In [ ]:
#Register the API key and select device
api_key = ""
register_key(api_key)

#T4 runtime uses nvidia
device = "cuda"

In [ ]:
#Instantiate chroma
with contextlib.redirect_stdout(None):
    chroma = Chroma()

the 'LENGTH' parameter determines the amount of amino acids in the protein. The larger this number the more complex the protein structure will be.

The 'CHARACTER' parameter is the desired shape the protein will form.

In [ ]:
#Number of amino acids in protein
LENGTH = 200

#Letter to generate
CHARACTER = "A"

#Make sure the character is only 1
if len(CHARACTER) > 1:
    CHARACTER = CHARACTER[:1]
    print(f"First character was picked: ({CHARACTER})!")

#Turn letter in discrete set of data points in space
letter_point_cloud = letter_to_point_cloud(CHARACTER)

#ShapeConditioner
conditioner = conditioners.ShapeConditioner(
    letter_point_cloud,
    chroma.backbone_network.noise_schedule,
    autoscale_num_residues = LENGTH,
).to(device)

#Generate protein shape
letter_protein, trajectories = chroma.sample(
    chain_lengths=[LENGTH], conditioner=conditioner, full_output=True
)

#Save protein to pdb
letter_protein.to(f"{CHARACTER}_protein.pdb")
print(f"\n{CHARACTER} letter protein: {letter_protein}")

print(f"\nProtein has been generated!")

## Chroma citation
@Article{Chroma2023,
  author  = {Ingraham, John B. and Baranov, Max and Costello, Zak and Barber, Karl W. and Wang, Wujie and Ismail, Ahmed and Frappier, Vincent and Lord, Dana M. and Ng-Thow-Hing, Christopher and Van Vlack, Erik R. and Tie, Shan and Xue, Vincent and Cowles, Sarah C. and Leung, Alan and Rodrigues, Jo\~{a}o V. and Morales-Perez, Claudio L. and Ayoub, Alex M. and Green, Robin and Puentes, Katherine and Oplinger, Frank and Panwar, Nishant V. and Obermeyer, Fritz and Root, Adam R. and Beam, Andrew L. and Poelwijk, Frank J. and Grigoryan, Gevorg},
  journal = {Nature},
  title   = {Illuminating protein space with a programmable generative model},
  year    = {2023},
  volume  = {},
  number  = {},
  pages   = {},
  doi     = {10.1038/s41586-023-06728-8}
}